In [6]:
import time
import pandas as pd
import json
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient
from dotenv import dotenv_values
from dotenv import load_dotenv
from src.login import get_ws_client

In [18]:
# Load env and login to Workspace
load_dotenv(".env")
config = dotenv_values(".env")

# Enter details of your Azure Machine Learning workspace
subscription_id = config.get("SUBSCRIPTION_ID")
resource_group = config.get("RESOURCE_GROUP")
workspace = config.get("AZUREML_WORKSPACE_NAME")

In [8]:
credential = DefaultAzureCredential()
# Check if given credential can get token successfully.
credential.get_token("https://management.azure.com/.default")


workspace_ml_client = get_ws_client(
    credential, subscription_id, resource_group, workspace
)
print(workspace_ml_client)

MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x0000022B3678CF10>,
         subscription_id=5a8ec57c-47f9-4bc3-aee5-9e4db1b89345,
         resource_group_name=olonok-ml,
         workspace_name=azure-ml)


In [9]:
# read ./emotion-dataset/small_test.jsonl into a pandas dataframe
test_df = pd.read_json("./emotion-dataset/small_test.jsonl", lines=True)
# take 5 random samples
test_df = test_df.sample(n=5)
# rebuild index
test_df.reset_index(drop=True, inplace=True)
# rename the label_string column to ground_truth_label
test_df = test_df.rename(columns={"label_string": "ground_truth_label"})
test_df.head(5)

,text,label,ground_truth_label
0,i feel his gracious presence even now,1,joy
1,i had the feeling that it might not have been ...,1,joy
2,im feeling kind of melancholy and really want ...,0,sadness
3,im feeling greedy for right now,3,anger
4,i dont know how to deal with this i feel like ...,4,fear


In [13]:
# create a json object with the key as "inputs" and value as a list of values from the text column of the test dataframe
test_df_copy = test_df[["text"]]
test_df_copy = test_df_copy.rename(columns={"text": "input_string"})
test_json = {"input_data": test_df_copy.to_dict("split")}
# save the json object to a file named sample_score.json in the ./emotion-dataset folder
with open("./emotion-dataset/sample_score.json", "w") as f:
    json.dump(test_json, f)

In [15]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) 

In [22]:
data= {
  "input_data": [
    ["i feel his gracious presence even now"], 
      ["i had the feeling that it might not have been taken as the truthful and sincere compliment it would have been"], 
      ["im feeling kind of melancholy and really want to go home and cuddle up with my boys"], 
      ["im feeling greedy for right now"], 
      ["i dont know how to deal with this i feel like its becoming apart if who i am im afraid that im going to associate it with regular things so that i will never forget it"]
  ],
  "params": {
    "return_all_scores": True
  }
}
body = str.encode(json.dumps(data))

In [20]:
url = 'https://emotion-1710679378.uksouth.inference.ml.azure.com/score'
# Replace this with the primary/secondary key or AMLToken for the endpoint
api_key = config.get("KEY")
if not api_key:
    raise Exception("A key should be provided to invoke the endpoint")

In [23]:
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key), 'azureml-model-deployment': 'demo' }

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))

b'[{"0": {"label": "anger", "score": 0.000270799791906029}, "1": {"label": "fear", "score": 0.00034690299071371555}, "2": {"label": "joy", "score": 0.8664152026176453}, "3": {"label": "love", "score": 0.1316552609205246}, "4": {"label": "sadness", "score": 0.0005942079587839544}, "5": {"label": "surprise", "score": 0.0007175387581810355}}, {"0": {"label": "anger", "score": 0.007055370137095451}, "1": {"label": "fear", "score": 0.0044976212084293365}, "2": {"label": "joy", "score": 0.9613637328147888}, "3": {"label": "love", "score": 0.02018970623612404}, "4": {"label": "sadness", "score": 0.0011905002174898982}, "5": {"label": "surprise", "score": 0.005703076720237732}}, {"0": {"label": "anger", "score": 0.0019255997613072395}, "1": {"label": "fear", "score": 0.00450942711904645}, "2": {"label": "joy", "score": 0.003267080755904317}, "3": {"label": "love", "score": 0.01176548469811678}, "4": {"label": "sadness", "score": 0.9784568548202515}, "5": {"label": "surprise", "score": 7.557063

In [25]:
result = eval(result.decode("utf8", 'ignore'))

In [34]:
for t, output in zip(data.get("input_data"), result):
    print(t[0])
    print("Prediction")
    for key in output.keys():
        print(output[key])
    print("\n")

i feel his gracious presence even now
Prediction
{'label': 'anger', 'score': 0.000270799791906029}
{'label': 'fear', 'score': 0.00034690299071371555}
{'label': 'joy', 'score': 0.8664152026176453}
{'label': 'love', 'score': 0.1316552609205246}
{'label': 'sadness', 'score': 0.0005942079587839544}
{'label': 'surprise', 'score': 0.0007175387581810355}


i had the feeling that it might not have been taken as the truthful and sincere compliment it would have been
Prediction
{'label': 'anger', 'score': 0.007055370137095451}
{'label': 'fear', 'score': 0.0044976212084293365}
{'label': 'joy', 'score': 0.9613637328147888}
{'label': 'love', 'score': 0.02018970623612404}
{'label': 'sadness', 'score': 0.0011905002174898982}
{'label': 'surprise', 'score': 0.005703076720237732}


im feeling kind of melancholy and really want to go home and cuddle up with my boys
Prediction
{'label': 'anger', 'score': 0.0019255997613072395}
{'label': 'fear', 'score': 0.00450942711904645}
{'label': 'joy', 'score': 0.003